In [1]:
## NAME:Devyani Mardia
## NETID: dm1633
## Group members: David & Devyani

################s
## Code for HW 3 problem 2
##
## INSTRUCTIONS:
## The following file uses ridge regression and logistic ridge regression to
## predict the daily log-return of J.P. Morgan stocks.
##
## For part a, complete all parts labeled FILL IN.
##
################


import pandas as pd
import numpy as np

stocks = pd.read_csv("sp500_long.csv")
stocks = stocks[["BAC", "JPM", "GOOGL", "C", "GS"]]

ndays = stocks.shape[0]
lag = 7

num_comp = stocks.shape[1]

stocks = np.array(stocks)
Y = stocks[lag:ndays, 1]

## X is a matrix of dimension   (ndays-lag) x (num_comp*lag+1)
X = np.ones((ndays-lag, num_comp*lag+1))
for ell in range(lag):
    X[:, (num_comp*ell):(num_comp*(ell+1))] = stocks[ell:(ndays-lag+ell), :]

ntrain = 700
nvalid = 200

train_ixs = np.arange(ntrain)
valid_ixs = np.arange(ntrain, ntrain+nvalid)
test_ixs = np.arange(ntrain+nvalid, len(Y))

Ytrain = Y[train_ixs]
Xtrain = X[train_ixs, :]

Yvalid = Y[valid_ixs]
Xvalid = X[valid_ixs, :]

Ytest = Y[test_ixs]
Xtest = X[test_ixs, :]


In [2]:
p = X.shape[1]

lambdas = np.power(float(2), np.arange(-10, 12.5, 0.5))

valid_errs = np.zeros(len(lambdas))

for il in range(len(lambdas)):
    betahat = np.linalg.inv(Xtrain.T @ Xtrain + (il * np.identity(36))) @ Xtrain.T @ Ytrain
    valid_errs[il] = np.sum((Yvalid - Xvalid @ betahat)**2)

lambda_star = lambdas[np.argmin(valid_errs)]

Xlearn = np.vstack((Xtrain, Xvalid))
Ylearn = np.concatenate((Ytrain, Yvalid))
betahat = np.linalg.inv(Xtrain.T @ Xtrain + (lambda_star * np.identity(36))) @ Xtrain.T @ Ytrain
## FILL IN: compute ridge regression estimate on combined training and validation data
          ## with the optimal lambda chosen by validation

test_err = np.sqrt(np.mean((Ytest - Xtest @ betahat)**2) )
baseline_err = np.sqrt(np.mean((Ytest - np.mean(Ylearn))**2))

print(f"Ridge test error: {test_err:.6f} Baseline error: {baseline_err:.6f}")

Ridge test error: 0.008380 Baseline error: 0.008530


Part b (free response)
Why can't we use cross-validation for this prediction problem?
Although we do perform some validation to select an optimal value for lambda, we do not – as we did in problem 1 – perform cross-validation. This is be-cause in this problem, we are working with time se-ries data, so sampling from our full dataset is not as straightforward. There are several additional factors that complicate cross validation given the nature of time series data, including things that could cause poor model behavior (e.g., points could be temporal-ly clumped or drawn in awkward orientations to cy-clic behavior in the data). There is also a potential theoretical “violation” when we apply straightfor-ward cross-validation to time series data, as we are allowing the model to peek into the future when generating a model; in other words, we could be us-ing future data to predict backwards, which is not the intention of a predictive model. There are some techniques that we could employ that consider the nature of time series data, such as sliding window cross validation or blocked cross validation.


Part c (free response)
Increase the value of the \lag" variable to 10, then to 15, 20, 25, and 30. What e ect does this have
on the covariates Xi? What e ect does this have on both the in-sample and the test regression
errors?

Solution:

For each increase in the lag variable, we are includ-ing an additional set of historical data in the covari-ate matrix we are using to train the model. Consider the case when our lag is 5. When we create a covari-ate matrix for our model, its first “observation” will be created using the first 5 observations of the origi-nal data, and it’s last observation will use the final 5 observations from the original data. We will have covariates with a lot of overlap, and even more as we increase the size of lag. In doing so, we use original observations to create a covariate matrix that be-haves like sliding a window of size lag down the da-taset, grabbing all the covariate information and the final response value for each time the window moves.
This is helpful, since it allows us to capture historical data and help our model pay attention to trends in the data. However, we pay the price of quite quickly bloating our covariate size (and doing so without including any additional information, only repeating values), so we run an extreme risk of overfitting. We see this explicitly as w e vary the lag value in our stock prediction model, with our baseline error re-maining consistent as our test error grows, show in the figure.